In [42]:
import advent
advent.scrape(2015, 24)
data: list[int] = advent.get_lines(24, map_fn=int)

all_nums = set(data)

# We know that each bag must weigh 516 since that is a third of the total
bagsize = sum(data) // 3

In [43]:
# Well let's just try some dynamic programming I guess...
from functools import cache

@cache
def grab(bag, bagsize) -> tuple[tuple[int, ...], ...]:
    # returns a list of tuples a1, a2, ...
    # where sum(ai) == bagsize
    # to help caching, bag/bagsize should be sorted
    if bagsize == 0: return ((),) # man this is so counterintuitive
    result = []
    for ix in range(len(bag)):
        if bag[ix] > bagsize: continue
        newbag = bag[:ix] + bag[(ix+1):]
        for r in grab(newbag, bagsize-bag[ix]):
            r = r + (bag[ix],)
            result.append(r)
    return tuple(result)

#print(grab(tuple([1,2,3,4,5,7,8,9,10,11]), 20))

def product(l) -> int:
    r = 1
    for i in l: r *= i
    return r

# Runs too long :(
# all_bags_mine = grab(tuple(data), bagsize)

In [44]:
from collections import defaultdict

# Well I have to admit...
# This 'find_combinations_dp' function is copied from online
# It should do pretty much the same thing as grab, but somehow is way faster...
# Other than that, it does pretty much the same as the grab function but with a
# dict and without recursion rather than @cache, so not sure why its so much faster
# maybe because it doesn't do the stupid tuple re-allocating

def find_combinations_dp(nums: list[int], target: int) -> list[list[int]]:
    dp = defaultdict(list)
    dp[0].append([])  # Base case: one way to sum to 0
    
    for num in nums:
        for t in range(target, num - 1, -1):
            for combination in dp[t - num]:
                dp[t].append(combination + [num])
    
    return dp[target]

all_bags = find_combinations_dp( data, bagsize)

In [45]:
possibilities = []
for bag in all_bags:
    possibilities.append(((len(bag), product(bag)), bag))

possibilities = sorted(possibilities)
for score, bag in possibilities:
    leftover = list(all_nums - set(bag))
    possible = find_combinations_dp(leftover, bagsize)
    if possible:
        # Since possibilities is sorted, this one is guaranteed to be smallest
        print(score[1])
        break

11266889531


In [ ]:
# Part 2
bagsize2 = sum(data) // 4

# Much quicker because bagsize2 is so much lower
all_bags_part2 = find_combinations_dp( data, bagsize2)

In [50]:
possibilities2 = []
for bag in all_bags_part2:
    possibilities2.append(((len(bag), product(bag)), bag))

possibilities2 = sorted(possibilities2)
for score, bag in possibilities2:
    leftover = list(all_nums - set(bag))
    possible = find_combinations_dp(leftover, bagsize2)
    for wecangodeeper in possible:
        leftover2 = list(set(leftover) - set(wecangodeeper))
        actuallypossible = find_combinations_dp(leftover2, bagsize2)
        if actuallypossible:
            print(score[1])
            break
    break # breaks out already during the first iteration but oh well it works
# Turns out it already succeeds during the very first one

77387711
